# GatorTronS

## Train the model

In [ ]:
import logging
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere

%run /home/ec2-user/SageMaker/LLM-NER-clinical-text/src/models/train_model.py \
--model_name 'UFNLP/gatortrons' \
--data_dir 'ncbi_disease' \
--batch_size 4 \
--num_train_epochs 15 \
--learning_rate 1e-5 \
--weight_decay 0.0 \
--new_model_dir "/home/ec2-user/SageMaker/LLM-NER-clinical-text/models/ncbi-disease/gatortrons/" \
--path_umls_semtype '/home/ec2-user/SageMaker/LLM-NER-clinical-text/data/public/MedMentions/SemGroups_2018.txt'


Loading and preprocessing the dataset ...
ncbi_disease
The device to run the model: cuda
Load the pretrained model ...


/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/Tutorial-LLM/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of MegatronBertForTokenClassification were not initialized from the model checkpoint at UFNLP/gatortrons and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The model has 354.221059millions parameters.


Epoch,Training Loss,Validation Loss,F1
1,0.069200,0.042130,0.832815


## Evaluate on test data


In [4]:
from src.models.model import *

# Load the model
model_loader = ModelLoader('/home/ec2-user/SageMaker/LLM-NER-clinical-text/models/ncbi-disease/gatortrons/checkpoint-19026/')

# Evaluate the models on metrics
scores, prediction_results, classmap = model_loader.evaluate_model(dataset_name='ncbi_disease',\
                                     path_umls_semtype='../data/public/MedMentions/SemGroups_2018.txt',\
                                     metric_names=['f1', 'precision', 'recall'])

scores

Loading and preprocessing the dataset ...
ncbi_disease


Map:   0%|          | 0/5433 [00:00<?, ? examples/s]

Map:   0%|          | 0/924 [00:00<?, ? examples/s]

Map:   0%|          | 0/941 [00:00<?, ? examples/s]

Map:   0%|          | 0/941 [00:00<?, ? examples/s]

{'f1': 0.876008064516129,
 'precision': 0.9052083333333333,
 'recall': 0.8486328125}

## Fix the id2label in config file

In [6]:
import json
from src.data.data_loader import *
from huggingface_hub import HfApi

# Load the config file
f = open('../models/ncbi-disease/gatortrons/config.json')
config = json.load(f)

# Load the dataset with label mapping
dataset_loader = DatasetLoader(dataset_name='ncbi_disease', model_name='/home/ec2-user/SageMaker/LLM-NER-clinical-text/models/ncbi-disease/gatortrons/', \
                               path_umls_semtype='../data/public/MedMentions/SemGroups_2018.txt')
dataset, classmap, umls_label_code, _ = dataset_loader.load_dataset()

# Add the label to the config
id_to_label = {ind: classmap.int2str(ind) for ind in range(len(classmap.names))}
config['id2label'] = id_to_label
config['label2id'] = {id_to_label[key]: key for key in id_to_label.keys()}

# Save the config
with open('../models/ncbi-disease/gatortrons/config.json', 'w') as f:
    json.dump(config, f)


Loading and preprocessing the dataset ...
ncbi_disease


In [7]:
# Push the config to hub
api = HfApi()
api.upload_file(
    path_or_fileobj="../models/ncbi-disease/gatortrons/config.json",
    path_in_repo="config.json",
    repo_id="longluu/Clinical-NER-NCBI-Disease-GatorTronS",
    repo_type="model",
    commit_message="fix the label2id and id2label in config"
)

CommitInfo(commit_url='https://huggingface.co/longluu/Clinical-NER-NCBI-Disease-GatorTronS/commit/735a2187ede117fb2a8a6f2d93ca56d969473752', commit_message='fix the label2id and id2label in config', commit_description='', oid='735a2187ede117fb2a8a6f2d93ca56d969473752', pr_url=None, pr_revision=None, pr_num=None)

## Push model to hub

In [4]:
from huggingface_hub import notebook_login

notebook_login()


In [5]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer

# Load the model
tokenizer = AutoTokenizer.from_pretrained('/home/ec2-user/SageMaker/LLM-NER-clinical-text/models/ncbi-disease/gatortrons/')
NER_model = AutoModelForTokenClassification.from_pretrained('/home/ec2-user/SageMaker/LLM-NER-clinical-text/models/ncbi-disease/gatortrons/checkpoint-19026/')

# Push the model to hub
NER_model.push_to_hub("longluu/Clinical-NER-NCBI-Disease-GatorTronS", commit_message='--batch_size 4 --num_train_epochs 15 --learning_rate 1e-5 --weight_decay 0.0')
tokenizer.push_to_hub("longluu/Clinical-NER-NCBI-Disease-GatorTronS", commit_message='--batch_size 4 --num_train_epochs 15 --learning_rate 1e-5 --weight_decay 0.0')

README.md:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/longluu/Clinical-NER-NCBI-Disease-GatorTronS/commit/c1614215c8dc0225c0264f39bbe438fa1120d859', commit_message='--batch_size 4 --num_train_epochs 15 --learning_rate 1e-5 --weight_decay 0.0', commit_description='', oid='c1614215c8dc0225c0264f39bbe438fa1120d859', pr_url=None, pr_revision=None, pr_num=None)